Data Partition as Training and Test sets

In [1]:
import os, sys
import image_norm_test as myData;
import numpy as np
from sklearn.cross_validation import train_test_split
seed = 3071986   # set random seed
rng = np.random.RandomState(seed)
import re

# fetch all the file in the directory path  defined#
pwd = os.pardir;
accessPath = r"%s\patch_database" %pwd;
filelist = []
print(accessPath)

def retrive_lable(filename):
    new_name = filename.split('\\')[2].split('_')[0]
    sample = int(re.findall(r'\d+', new_name)[0])
    return sample
    
def fetchFile(accessPath , numOfimageOfeachSample = 91):
    filelist = []
    lable = []
    index = 0
    i = 0
    for file in os.listdir(accessPath):
        if(index % 92 == i):
            completeName = os.path.join(accessPath, file)  
            filelist.append(completeName)
            lable.append(retrive_lable(completeName))
            if(i < numOfimageOfeachSample-1):  
                i+=1
            else:
                i = 0
        index +=1
    return filelist , lable

# in this way we can extract only certain number image of each samples, ex. 2 means only 2 images of one texture
filelist , lable = fetchFile(accessPath , 4)
print(filelist[0])
print(filelist[100])
filelist = np.array(filelist).transpose();
lable = np.array(lable).transpose();
print(lable)


..\patch_database
..\patch_database\sample01_002_3x3
..\patch_database\sample26_002_3x3
[ 1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6  7
  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11 12 12 12 12 13 13
 13 13 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17 18 18 18 18 19 19 19
 19 20 20 20 20 21 21 21 21 22 22 22 22 23 23 23 23 24 24 24 24 25 25 25 25
 26 26 26 26 27 27 27 27 28 28 28 28 29 29 29 29 30 30 30 30 31 31 31 31 32
 32 32 32 33 33 33 33 34 34 34 34 35 35 35 35 36 36 36 36 37 37 37 37 38 38
 38 38 39 39 39 39 40 40 40 40 41 41 41 41 42 42 42 42 43 43 43 43 44 44 44
 44 45 45 45 45 46 46 46 46 47 47 47 47 48 48 48 48 49 49 49 49 50 50 50 50
 51 51 51 51 52 52 52 52 53 53 53 53 54 54 54 54 55 55 55 55 56 56 56 56 57
 57 57 57 58 58 58 58 59 59 59 59 60 60 60 60 61 61 61 61]


In [2]:
from sklearn import preprocessing


rng = np.random.RandomState(seed)
permutation = rng.permutation(len(filelist))

#print(permutation)
#print(filelist[permutation])

X, y = filelist[permutation], lable[permutation]


# split our data into half of train data and half of text data randomly.
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.5, random_state=seed)


print(train_y)
print(test_y)
#print(test)


[60 41 34  9 37 12 57 13 34 44  2 55  7 14 21 17 36  4 45 56  5 39 31 44 23
 27  1  5 51 28 45 37 37 53 10  7 15 24 55 29 41 12  9 31 55 58 15 61 34 39
 52 19 20 36 47 56 54 19 13  1 53 22  3  5 59 35 47 10  8  5 39 43 46 56  2
  8 55 42 19 24 22 18 33 52  4 38 17 29 35 58 30 21 30 49 25  7 32 40 46 35
 44 49  6 10 20 10 28 45 54 27 25 21 32 31 12 24 45 16  3 48 33 50]
[42 46 56 50 58 33 23 42 26 24 30 16 31 43 17 40 20  3 57  2 59 26 47 35 51
 41 11 18 40 49 22 54 29 15 16  8 50 27 14  6 28  6 26 13 22 16 48 25 25 32
 36 32 58 15  2  9 48  3  4 60 38 33 47 59 61 38 59 60 36  6 14 43 52 40 11
 19 23 49 39 30 53 23 57 37 42 51 54 43  7 21 46 51  1 17  1  4 28 48 34 29
 61 13 38 18 52 57 50 44 61  8 41 14 53 27 26 11 20 60 18 12  9 11]


Start to get the features learned from feed in data

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import time


def read_features(file_name):
    features = list()

    f = open(file_name)
    for line in f.readlines():
        feature_list = line.split(" ")
        feature = list()
        for item in feature_list:
            if item != '\n':
                feature.append(float(item))
        features.append(feature)

    f.close()
    return features


Retrive centroids ( features ) from file stored in disk

In [4]:
centroid = []
count = 0
temp = []
n = 3*3
num_data = 10000
clusters = [16,32,64,128]
#cluster = 16
#centroid_file = r"%s\%dfeature_%d.txt" %(pwd ,  cluster , num_data);
for cluster in clusters:
    centroid_file = r"%s\25_4_3_%d.txt" %(pwd , cluster)
    centroid.append(read_features(centroid_file))
'''
data = np.loadtxt(centroid_file,delimiter="\n")
for i in range(0 , len(data)):
    temp.append(data[i])
    count += 1
    if(count == n):
        centroid.append(temp)
        temp = []
        count = 0
'''
print(centroid[0])
print(len(centroid[0]))

[[-0.359201245835, -0.350276323095, -0.284874560613, -0.387853317701, -0.356222505484, -0.265256232066, -0.382704706903, -0.336148081943, -0.244915257572], [0.358806866422, 0.290237064273, 0.279483142322, 0.341993511392, 0.295504326746, 0.315070116924, 0.35871147642, 0.356818418472, 0.386476904451], [-0.148784868061, 0.164467159988, 0.442070713355, -0.131500140521, 0.248641382045, 0.541956821763, -0.0695002573605, 0.294172120952, 0.539617973866], [-0.37811067742, -0.182680751809, 0.262972398401, -0.36602944445, 0.000716159993457, 0.476164862946, -0.207526841013, 0.192900624991, 0.55992543393], [0.211746974514, 0.364218791337, 0.373559949287, 0.261235304683, 0.405884763544, 0.384645076997, 0.275612693403, 0.367548912751, 0.301609240793], [0.49121356796, 0.349911768499, 0.0140163857025, 0.511247189891, 0.315396451391, -0.040035183695, 0.459491759478, 0.238122601938, -0.0760792258937], [-0.270118685166, 0.188059162761, 0.550179916507, -0.344981907231, -0.22451350832, 0.0962181922777, -0.2

Get the bag of features for each patch 

In [7]:
from my_vlad import my_vlad

# will load data as the patch size defined , 3 means 3*3 = 9 for each patch, and will return the dictionary included:
# 'data'  (one patch)  , 'target' (the sample of this patch belongs to ) , 'filename' (the file comes from)
bofs = []
vlad = my_vlad(centroid[0])
for file in train_X[0:10]:
    mydata,y = myData.load_sig_data(file , 3)
    bofs.append(vlad.get_vlad(mydata['data']).flatten() )
#print(bofs[0])
#print(X_train_tf)
#print(train_y)


40000
40000
40000
40000
40000
40000
40000
40000
40000
40000


Used the bag of features of each image and the known lable to do classification with knn 

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import grid_search


#knn_init = KNeighborsClassifier()
#parameters = {'n_neighbors':[ 5, 10 , 15]}
#knn = grid_search.GridSearchCV(knn_init, parameters)

knn = KNeighborsClassifier(n_neighbors = 2)

knn.fit(bofs, train_y[0:10])

C:\Users\howfungirl\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.19.
  DeprecationWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [10]:
bofs_test = []
for data in test_X[0:10]:
    mydata,y = myData.load_sig_data(file , 3)
    bofs_test.append(vlad.get_vlad(mydata['data']).flatten() )
#print(bofs)


40000
40000
40000
40000
40000
40000
40000
40000
40000
40000


In [13]:
import numpy as np
predicted = knn.predict(bofs_test)
print(predicted)
print(test_y[0:10])
knn.score(bofs_test,test_y[0:10])

 
#np.mean(predicted == test_y)  

[34 34 34 34 34 34 34 34 34 34]
[42 46 56 50 58 33 23 42 26 24]


0.0